In [2]:
import xml.etree.ElementTree as ET
import subprocess
import os
import random

files = ["AdvTests/4768668.xml", "AdvTests/Starcraft.xml", "AdvTests/Mazes1.xml"]

In [ ]:
print("{:20} {:10} {:3} {:10} {:10} {:8}".format("File name", "Algo", "It", "Steps", "Node count", "Length"))
random.seed(1337)
for f in files:
    name = f.split("/")[-1].split(".")[0]
    tree = ET.parse(f)
    root = tree.getroot()
    l = [None]
    for i in range(20):
        w = int(tree.find("map").find("width").text)
        h = int(tree.find("map").find("height").text)
        while True:
            startx = random.randint(0, w // 5)
            starty = random.randint(0, h // 5)
            finishx = random.randint(w // 5 * 4, w - 1)
            finishy = random.randint(h // 5 * 4, h - 1)
            startrow = tree.find("map").find("grid")[starty].text.split(" ")
            finishrow = tree.find("map").find("grid")[finishy].text.split(" ")
            if startrow[startx] == "0" and finishrow[finishx] == "0":
                l.append([startx, starty, finishx, finishy])
                break
        
    for p in l:
        if p:
            tree.find("map").find("startx").text = str(p[0])
            tree.find("map").find("starty").text = str(p[1])
            tree.find("map").find("finishx").text = str(p[2])
            tree.find("map").find("finishy").text = str(p[3])
        arastar_in = "tmp/" + name + "_arastar.xml"
        arastar_out = "tmp/" + name + "_arastar_log.xml"
        root.find("algorithm").find("searchtype").text = "arastar"
        root.find("algorithm").find("hweight").text = "3"
        if root.find("algorithm").find("hstep") == None:
            ET.SubElement(root.find("algorithm"), "hstep")
        root.find("algorithm").find("hstep").text = "0.2"
        tree.write(arastar_in)
        subprocess.run(args=["./cmake-build-debug/Src/ARAStarPathPlanning", arastar_in])
        out_tree = ET.parse(arastar_out)
        out_root = out_tree.getroot()
        res = out_root.find("log").findall("result")
        val = -1
        for i, e in enumerate(res):
            summ = e.find("summary")
            print("{:20} {:10} {:3} {:10} {:10} {:8}".format(name, "arastar", i, summ.attrib["numberofsteps"], summ.attrib["nodescreated"], e.find("summary").attrib["length"]))
            if val == -1 and i != 0 and summ.attrib["numberofsteps"] != "0":
                val = float(e.find("summary").attrib["value"])

        root.find("algorithm").find("searchtype").text = "astar"
        root.find("algorithm").find("hweight").text = "1"
        astar_in = "tmp/" + name + "_astar.xml"
        astar_out = "tmp/" + name + "_astar_log.xml"
        tree.write(astar_in)
        subprocess.run(args=["./cmake-build-debug/Src/ARAStarPathPlanning", astar_in])
        out_tree = ET.parse(astar_out)
        out_root = out_tree.getroot()
        summ = out_root.find("log").find("result").find("summary")
        print("{:20} {:10} {:3} {:10} {:10} {:8}".format(name, "astar", 0, summ.attrib["numberofsteps"], summ.attrib["nodescreated"], summ.attrib["length"]))
        if val != -1:
            root.find("algorithm").find("searchtype").text = "astar"
            root.find("algorithm").find("hweight").text = str(val)
            wastar_in = "tmp/" + name + "_wastar.xml"
            wastar_out = "tmp/" + name + "_wastar_log.xml"
            tree.write(wastar_in)
            subprocess.run(args=["./cmake-build-debug/Src/ARAStarPathPlanning", wastar_in])
            out_tree = ET.parse(wastar_out)
            out_root = out_tree.getroot()
            summ = out_root.find("log").find("result").find("summary")
            print("{:20} {:10} {:3} {:10} {:10} {:8}".format(name, "wastar", val, summ.attrib["numberofsteps"], summ.attrib["nodescreated"], summ.attrib["length"]))

        print("")
    
    
    

File name            Algo       It  Steps      Node count Length  
4768668              arastar      0 2386       3282       708     
4768668              arastar      1 0          2223       708     
4768668              arastar      2 0          2223       708     
4768668              arastar      3 0          2223       708     
4768668              arastar      4 15796      16726      708     
4768668              arastar      5 40338      40992      628     
4768668              astar        0 40338      40905      628     
4768668              wastar     1.4 13519      14305      656     

4768668              arastar      0 1642       2531       851     
4768668              arastar      1 0          1598       851     
4768668              arastar      2 0          1598       851     
4768668              arastar      3 0          1598       851     
4768668              arastar      4 0          1598       851     
4768668              arastar      5 2892       4592       767

4768668              astar        0 1187       2026       693     
4768668              wastar     1.0 1187       2026       693     

4768668              arastar      0 1381       2458       945     
4768668              arastar      1 0          1323       945     
4768668              arastar      2 0          1323       945     
4768668              arastar      3 0          1323       945     
4768668              arastar      4 22         1323       945     
4768668              arastar      5 14360      15744      837     
4768668              astar        0 14571      15937      837     
4768668              wastar     1.4 2230       3302       883     

4768668              arastar      0 1783       2850       835     
4768668              arastar      1 0          1728       835     
4768668              arastar      2 0          1728       835     
4768668              arastar      3 0          1728       835     
4768668              arastar      4 629        1854       83